# Assignment II: Discrete Event Simulation

Dependencies:
- Numpy
- Simpy
- Matplotlib

In [1]:
import numpy as np
import simpy
import matplotlib.pyplot as plt
import random

In [2]:
# Based on https://simpy.readthedocs.io/en/latest/examples/carwash.html

RANDOM = 1
SERVICE_TIME = 4
SIM_TIME = 50

class ServerUsage(object):
    def __init__(self, env, num_machines, service_time):
        self.env = env
        self.machines = simpy.Resource(env, num_machines)
        self.service_time = service_time

    def service(self, customer):
        yield self.env.timeout(SERVICE_TIME)
        print(f'Customer {customer} has been helped')

def customer(env, name, Server):

    print(f"Customer {name} arrives at {env.now}")
    with Server.machines.request() as request:
        
        yield request
       
        print(f"Customer {name} enters service at {env.now}")
        yield env.process(Server.service(name))
        
        print(f"Customer {name} leaves service at {env.now}")

def setup(env, num_machines, service_time, t_inter):

    ServerEnv = ServerUsage(env, num_machines, service_time)

    for i in range(4):
        env.process(customer(env, i, ServerEnv))

    # Create more cars while the simulation is running
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(customer(env, 'Car %d' % i, ServerEnv))

random.seed(RANDOM)
env = simpy.Environment()
env.process(setup(env, num_machines=1, service_time=10, t_inter=5))
env.run(until=SIM_TIME)





Customer 0 arrives at 0
Customer 1 arrives at 0
Customer 2 arrives at 0
Customer 3 arrives at 0
Customer 0 enters service at 0


NameError: name 'SERVICE_TIME' is not defined